In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
price_query = session.sql("""
    SELECT * 
    FROM "DIAMONDS_DB"."DIAMONDPRICES_SCHEMA"."PRICE_DETAILS"
    LIMIT 1000
""")


In [ ]:
df = price_query.to_pandas()
df.head()

In [ ]:
df = df.drop(columns=["INDEX"])

df = df.reset_index(drop=True)

df.head()

In [ ]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [ ]:
# 1. Feature Engineering
df["CARAT_SQ"] = df["CARAT"] ** 2
df["VOLUME"] = df["X"] * df["Y"] * df["Z"]

# 2. Remove invalid volume rows
df = df[(df["VOLUME"] > 0.1) & (df["X"] > 0.1) & (df["Y"] > 0.1) & (df["Z"] > 0.1)]

# 3. Encode categorical features
encoder = OrdinalEncoder()
df[["CUT", "COLOR", "CLARITY"]] = encoder.fit_transform(df[["CUT", "COLOR", "CLARITY"]])

# 4. Remove top 1% PRICE outliers
q99 = df["PRICE"].quantile(0.99)
df = df[df["PRICE"] < q99]

# 5. Select features and target
features = [
    "CARAT", "CUT", "COLOR", "CLARITY", "DEPTH", "TABLES",
    "CARAT_SQ", "VOLUME", "X", "Y", "Z"
]
X = df[features]
y = df["PRICE"]

# 6. Train/test split and log-transform target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)


# 7. Normalize numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 8. Train models on scaled data
rf_model = RandomForestRegressor(
    n_estimators=250, max_depth=12, min_samples_split=4,
    max_features='sqrt', random_state=42, n_jobs=1
)
lgbm_model = LGBMRegressor(
    n_estimators=300, learning_rate=0.05, num_leaves=31,
    max_depth=6, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=1
)

rf_model.fit(X_train_scaled, y_train_log)
lgbm_model.fit(X_train_scaled, y_train_log)

# 9. Predict and reverse log transformation
rf_pred = np.expm1(rf_model.predict(X_test_scaled))
lgbm_pred = np.expm1(lgbm_model.predict(X_test_scaled))
y_true = np.expm1(y_test_log)

# 10. Evaluate
results = []
for name, pred in zip(["Random Forest", "LightGBM"], [rf_pred, lgbm_pred]):
    rmse = mean_squared_error(y_true, pred, squared=False)
    r2 = r2_score(y_true, pred)
    results.append({"Model": name, "RMSE": round(rmse, 2), "R² Score": round(r2, 4)})

results_df = pd.DataFrame(results).sort_values(by="RMSE")
print("Model Evaluation Results (with normalization):")
print(results_df)


In [ ]:
#train on the features and will evaluate on next step

# 1. Feature Engineering
df["CARAT_SQ"] = df["CARAT"] ** 2
df["VOLUME"] = df["X"] * df["Y"] * df["Z"]

# 2. Remove invalid volume rows
df = df[(df["VOLUME"] > 0.1) & (df["X"] > 0.1) & (df["Y"] > 0.1) & (df["Z"] > 0.1)]

# 3. Encode categoricals
encoder = OrdinalEncoder()
df[["CUT", "COLOR", "CLARITY"]] = encoder.fit_transform(df[["CUT", "COLOR", "CLARITY"]])

# 4. Remove top 1% PRICE outliers
q99 = df["PRICE"].quantile(0.99)
df = df[df["PRICE"] < q99]

# 5. Select features and log-transformed target
features = [
    "CARAT", "CUT", "COLOR", "CLARITY", "DEPTH", "TABLES",
    "CARAT_SQ", "VOLUME", "X", "Y", "Z"
]
X = df[features]
y = df["PRICE"]
y_log = np.log1p(y)  # Log-transform full target

# 6. Normalize full dataset (no split)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. Train models on entire dataset
rf_model = RandomForestRegressor(
    n_estimators=250, max_depth=12, min_samples_split=4,
    max_features='sqrt', random_state=42, n_jobs=1
)
lgbm_model = LGBMRegressor(
    n_estimators=300, learning_rate=0.05, num_leaves=31,
    max_depth=6, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=1
)

rf_model.fit(X_scaled, y_log)
lgbm_model.fit(X_scaled, y_log)

print("Models trained successfully on full dataset (log-transformed + normalized).")


In [ ]:
# Predict using trained models
rf_pred_log = rf_model.predict(X_scaled)
lgbm_pred_log = lgbm_model.predict(X_scaled)

# Reverse log transform predictions and target
rf_pred = np.expm1(rf_pred_log)
lgbm_pred = np.expm1(lgbm_pred_log)
y_actual = np.expm1(y_log)

# Evaluate model performance
results = []
for name, pred in zip(["Random Forest", "LightGBM"], [rf_pred, lgbm_pred]):
    rmse = mean_squared_error(y_actual, pred, squared=False)
    r2 = r2_score(y_actual, pred)
    results.append({
        "Model": name,
        "RMSE": round(rmse, 2),
        "R² Score": round(r2, 4)
    })

# Display results
results_df = pd.DataFrame(results).sort_values(by="RMSE")
print("Final Evaluation Results:\n")
print(results_df.to_string(index=False))


In [ ]:
rf_pred_log = rf_model.predict(X_scaled)
lgbm_pred_log = lgbm_model.predict(X_scaled)

rf_pred = np.expm1(rf_pred_log)
lgbm_pred = np.expm1(lgbm_pred_log)
y_actual = np.expm1(y_log)


In [ ]:
# 1. Actual vs Predicted Scatter Plot for both models
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_actual, y=rf_pred, alpha=0.5, label="Random Forest", color="steelblue")
sns.scatterplot(x=y_actual, y=lgbm_pred, alpha=0.5, label="LightGBM", color="darkorange")
sns.lineplot(x=y_actual, y=y_actual, color="black", linestyle="--", label="Ideal Fit")
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted Prices (Random Forest vs LightGBM)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Residual Plot Comparison
rf_residuals = y_actual - rf_pred
lgbm_residuals = y_actual - lgbm_pred

plt.figure(figsize=(10, 5))
sns.kdeplot(rf_residuals, label="Random Forest", fill=True, color="steelblue")
sns.kdeplot(lgbm_residuals, label="LightGBM", fill=True, color="darkorange")
plt.axvline(0, color="black", linestyle="--")
plt.title("Residual Distribution Comparison")
plt.xlabel("Prediction Error")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 3. Side-by-side Feature Importances
rf_importances = rf_model.feature_importances_
lgbm_importances = lgbm_model.feature_importances_

feature_df = pd.DataFrame({
    "Feature": X.columns,
    "Random Forest": rf_importances,
    "LightGBM": lgbm_importances
}).set_index("Feature")

feature_df.plot(kind="barh", figsize=(10, 6), color=["steelblue", "darkorange"])
plt.title("Feature Importances (Random Forest vs LightGBM)")
plt.xlabel("Importance Score")
plt.grid(True)
plt.tight_layout()
plt.show()
